In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

 99% 170M/172M [00:03<00:00, 43.6MB/s]
100% 172M/172M [00:03<00:00, 55.3MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [6]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [7]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [8]:
input = (256, 256, 3)

In [9]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [10]:
# Creating the base model
base_model = Xception(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [11]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, None, None, 2048   20861480  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21386281 (81.58 MB)
Trainable params: 524801 (2.00 MB)
Non-trainable params: 20861480 (79.58 MB)
_________________________________________________________________


In [14]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
history = model.fit(x_train,epochs=10, batch_size=100, validation_data=x_test)

Epoch 1/10
89/89 [==============================] - 73s 622ms/step - loss: 0.4972 - accuracy: 0.7543 - val_loss: 0.5390 - val_accuracy: 0.6933
Epoch 2/10
89/89 [==============================] - 52s 575ms/step - loss: 0.2876 - accuracy: 0.8925 - val_loss: 0.5470 - val_accuracy: 0.7267
Epoch 3/10
89/89 [==============================] - 51s 567ms/step - loss: 0.1915 - accuracy: 0.9405 - val_loss: 0.5517 - val_accuracy: 0.7167
Epoch 4/10
89/89 [==============================] - 53s 583ms/step - loss: 0.1346 - accuracy: 0.9621 - val_loss: 0.7271 - val_accuracy: 0.7083
Epoch 5/10
89/89 [==============================] - 50s 554ms/step - loss: 0.0987 - accuracy: 0.9728 - val_loss: 0.5997 - val_accuracy: 0.7200
Epoch 6/10
89/89 [==============================] - 52s 577ms/step - loss: 0.0674 - accuracy: 0.9845 - val_loss: 0.6347 - val_accuracy: 0.7267
Epoch 7/10
89/89 [==============================] - 52s 572ms/step - loss: 0.0511 - accuracy: 0.9902 - val_loss: 0.6724 - val_accuracy: 0.7267